<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/ENGULF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

bist30_stocks = [
    'AEFES.IS', 'AGHOL.IS', 'AKBNK.IS', 'AKCNS.IS', 'AKENR.IS', 'AKFGY.IS',
    'AKGRT.IS', 'AKSA.IS', 'AKSEN.IS', 'ALARK.IS', 'ARCLK.IS', 'ASELS.IS',
    'BIMAS.IS', 'DOHOL.IS', 'EKGYO.IS', 'ENJSA.IS', 'EREGL.IS', 'FROTO.IS',
    'GUBRF.IS', 'HEKTS.IS', 'ISCTR.IS', 'KCHOL.IS', 'KOZAA.IS', 'KOZAL.IS',
    'KRDMD.IS', 'ODAS.IS', 'PETKM.IS', 'PGSUS.IS', 'SAHOL.IS', 'SASA.IS',
    'TUPRS.IS'
]

def fetch_stock_data(ticker, period='3mo', interval='1d'):
    stock_data = yf.download(ticker, period=period, interval=interval)
    stock_data['EMA5'] = stock_data['Close'].ewm(span=5, adjust=False).mean()
    stock_data['EMA20'] = stock_data['Close'].ewm(span=20, adjust=False).mean()
    return stock_data[['Open', 'High', 'Low', 'Close', 'EMA5', 'EMA20']]


def detect_engulfing(df):

    conditions_bullish = (
        (df['Open'].shift(1) > df['Close'].shift(1)) &
        (df['Open'] < df['Close']) &
        (df['Open'] < df['Close'].shift(1)) &
        (df['Close'] > df['Open'].shift(1))
    )


    conditions_bearish = (
        (df['Open'].shift(1) < df['Close'].shift(1)) &
        (df['Open'] > df['Close']) &
        (df['Open'] > df['Close'].shift(1)) &
        (df['Close'] < df['Open'].shift(1))
    )

    df['SignalGreen'] = conditions_bullish
    df['SignalRed'] = conditions_bearish
    return df


for ticker in bist30_stocks:
    print(f"Analyzing {ticker}...")

    try:

        stock_data = fetch_stock_data(ticker)


        if stock_data.empty:
            print(f"No data for {ticker}. Skipping.")
            continue

        stock_data = detect_engulfing(stock_data)


        fig = go.Figure(data=[go.Candlestick(x=stock_data.index,
                        open=stock_data.Open, high=stock_data.High,
                        low=stock_data.Low, close=stock_data.Close)
                             ])

        fig.update_layout(xaxis_rangeslider_visible=False)
        fig.update_layout(
            width=1000, height=800,
            title=f"{ticker.replace('.IS', '')} Engulfing Pattern with EMA5 and EMA20",
            yaxis_title='Price',
            xaxis=dict(type="category"),
            xaxis_tickformat='%Y-%m-%d'  # Sadece tarih göster
        )


        fig.add_trace(go.Scatter(
            x=stock_data.index,
            y=stock_data['EMA5'],
            mode='lines',
            name='EMA5',
            line=dict(color='orange', width=2)
        ))


        fig.add_trace(go.Scatter(
            x=stock_data.index,
            y=stock_data['EMA20'],
            mode='lines',
            name='EMA20',
            line=dict(color='blue', width=2)
        ))


        if stock_data['SignalGreen'].any():
            fig.add_trace(go.Scatter(
                x=stock_data.index[stock_data['SignalGreen']],
                y=stock_data['High'][stock_data['SignalGreen']] + 10,
                name='Engulf Green',
                mode="markers",
                marker_size=10,
                marker=dict(color='green', symbol="square-dot")
            ))

        if stock_data['SignalRed'].any():
            fig.add_trace(go.Scatter(
                x=stock_data.index[stock_data['SignalRed']],
                y=stock_data['High'][stock_data['SignalRed']] + 10,
                name='Engulf Red',
                mode="markers",
                marker_size=10,
                marker=dict(color='red', symbol="square-dot")
            ))


        fig.show()

    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")
